<a href="https://colab.research.google.com/github/YATHAGROVER/ML/blob/main/Student_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pypdf
!pip  install -q transformers einops accelerate langchain bitsandbytes sentence_transformers llama_index llama-index-llms-huggingface gradio pytube jiwer ReportLab weasyprint
!pip install -q -i https://pypi.org/simple/ bitsandbytes
!pip install -q git+https://github.com/openai/whisper.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38

In [ ]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [ ]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [ ]:
# prompt: make a code to download a file and the wright it in a file

import requests
import os
if not os.path.exists('data'):
  os.makedirs('data')

def download_and_write_file(url, filename):
  response = requests.get(url)

  if response.status_code == 200:
    with open(filename, 'wb') as f:
      f.write(response.content)
  else:
    print('Failed to download file.')


download_and_write_file('https://arxiv.org/pdf/1706.03762.pdf','data/attention is all you need.pdf')

In [ ]:
documents= SimpleDirectoryReader("/content/data").load_data()
# documents

ValueError: Directory /content/data does not exist.

In [ ]:
system_prompt="""
You are a Q&A assistant. your goal is to answer questions as
accurately as possible bassed on the instruction and context provoided.
"""
# Default format supportableby LLama 2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM

device = 'cuda' if torch.cuda.is_available() else "cpu"
llm= HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs= {"temperature":0,"do_sample":False},
    system_prompt= system_prompt,
    query_wrapper_prompt= query_wrapper_prompt,
    tokenizer_name= "google/gemma-1.1-2b-it",
    model_name= "google/gemma-1.1-2b-it",
    device_map= device,
    model_kwargs= {"torch_dtype": torch.float16, "quantization_config":BitsAndBytesConfig(
        load_in_4bit=True,  # Set to True for 4-bit quantization
        load_in_8bit=False,  # Set to True for 8-bit quantization
    )},
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-1.1-2b-it.
401 Client Error. (Request ID: Root=1-66a9b331-78480d191e94cf231fdc456f;d8558439-c8f4-481c-9298-482fa4e7fffb)

Cannot access gated repo for url https://huggingface.co/google/gemma-1.1-2b-it/resolve/main/config.json.
Access to model google/gemma-1.1-2b-it is restricted. You must be authenticated to access it.

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.7 MB/s eta 0:00:00


In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.legacy.embeddings.langchain import LangchainEmbedding

embed_model= LangchainEmbedding(
    HuggingFaceEmbeddings(model_name= "sentence-transformers/all-mpnet-base-v2")
)

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
service_context= ServiceContext.from_defaults(
    chunk_size=1024,
    llm= llm,
    embed_model= embed_model
)

<ipython-input-13-e13bf89a3d6d>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context= ServiceContext.from_defaults(


In [ ]:
import llama_index
from llama_index.core import VectorStoreIndex
from sentence_transformers import SentenceTransformer

In [ ]:
embed_model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
index= VectorStoreIndex.from_documents(documents,service_context= service_context)

In [ ]:
query_engine= index.as_query_engine()

In [ ]:
%%time
response= query_engine.query("what do you think about srk ?")

In [ ]:
print(response.response)

In [ ]:
import whisper
import gradio as gr
from pytube import YouTube
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from weasyprint import HTML

model = whisper.load_model("base")
globvar=0

def echo(message, history):
  if "https://yout" in message or  "https://www.youtube.com/" in message:
    try:
      yt = YouTube(message)
      global globvar
      yt.streams.filter(only_video=True, file_extension='mp4')
      stream = yt.streams.get_by_itag(139)
      stream.download('',f"GoogleImagen{globvar}.mp4")
      result = model.transcribe(f"GoogleImagen{globvar}.mp4")
      # create_pdf_in_colab("my_pdf", result['text'])
      cont = result['text']
      filename = f"data/youtube{globvar}.pdf"
      globvar=globvar+1
      # Create a new PDF canvas
      c = canvas.Canvas(filename)

      # Add title to the PDF
      c.setFont("Helvetica", 24)
      c.drawCentredString(300, 750, 'title')
      # Define margins for content placement
      margin_left = 50
      margin_top = 700
      margin_right = 50

      # Check if using text frame or individual lines
      use_text_frame = len(cont.splitlines()) > 10  # Adjust threshold for long content

      if use_text_frame:
        # Use text frame for long content
        frame_width = 6.5 * inch  # Adjust width
        frame_height = 4.0 * inch  # Adjust height
        x_pos = margin_left
        y_pos = margin_top

        # Create text frame
        frame = c.beginFrame(x_pos, y_pos, frame_width, frame_height)

        # Set font and line spacing for frame
        c.setFont("Helvetica", 12)  # Adjust font size
        c.setLineSpacing(1.0)  # Adjust line spacing

        # Add content to the frame
        c.drawString(0, 0, cont)  # Assuming your content is in a variable

        # End the text frame
        c.endFrame()
      else:
        # Use individual lines for short content
        c.setFont("Helvetica", 12)  # Adjust font size
        # c.setLineSpacing(1.0)  # Adjust line spacing (multiplies font size for spacing)

        y_pos = margin_top

        for line in cont.splitlines():
          c.drawString(margin_left, y_pos, line)
          y_pos -= 18  # Adjust line spacing based on font size

      # Save and close the PDF
      c.save()
      return "Video Uploaded"
    except:
      return "Please provide a valid Youtube link...."
  else:
    documents= SimpleDirectoryReader("/content/data").load_data()
    index= VectorStoreIndex.from_documents(documents,service_context= service_context)
    query_engine= index.as_query_engine()
    response = query_engine.query(message)
    return response.response

demo = gr.ChatInterface(fn=echo, examples=["what is the meaning of attention is all you need ?", "https://www.youtube.com/watch?v=u47GtXwePms", "https://youtu.be/u47GtXwePms?si=roRjj6cKfednKOUS"], title="GDSC AI BOT")
demo.launch(debug=False)

ModuleNotFoundError: No module named 'whisper'